In [1]:
import os
import json
import requests
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader


c:\Users\Vigneshwar Kandhaiya\Desktop\my learning\Agentic_ai\Ai_Agents_self_work\Agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
openai=OpenAI()

pushover_token=os.getenv("PUSHOVER_TOKEN")
pushover_user=os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

In [3]:
def push(message):
    print(f"Push: {message}")
    payload={"user": pushover_user,"token": pushover_token, "message": message}
    requests.post(pushover_url,data=payload)

In [4]:
push("Hi")

Push: Hi


In [5]:
def record_user_details(email,name="Name not provided",notes="Notes not provided"):
    push(f"Recording the interest from user {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}


In [6]:
def record_unknown_question(question):
    push(f"Recording question {question} that I could not answer")
    return {"recorded": "ok"}

In [7]:
record_user_details("vignesh.k2@gmail.com")

Push: Recording the interest from user Name not provided with email vignesh.k2@gmail.com and notes Notes not provided


{'recorded': 'ok'}

In [8]:
record_user_details_json={
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being touch and provide an email address",
    "parameters":{
        "type": "object",
        "properties":{
            "email":{
                "type": "string",
                "description": "The email address of the user"
            },
            "name":{
                "type": "string",
                "description": "The name of the user"
            },
            "notes":{
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }        
        },
        "required":["email"],
        "additionalProperties": False
    }
}

In [9]:
record_unknown_question_json={
    "name": "record_unknown_question",
    "description": "Always use this tool to record the questions that you are unable to answer as you do not have the context",
    "parameters":{
        "type": "object",
        "properties":{
            "question":{
                "type": "string",
                "description": "The question that could not be answered"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [10]:
tools=[
    {"type": "function","function": record_user_details_json},
    {"type": "function","function": record_unknown_question_json}
]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being touch and provide an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of the user'},
     'name': {'type': 'string', 'description': 'The name of the user'},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Always use this tool to record the questions that you are unable to answer as you do not have the context',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The question that could not be answered'}},
    'required': ['question'],
    'addition

In [21]:
def handle_tool_calls(tool_calls):
    results=[]
    for tool_call in tool_calls:
        tool_name=tool_call.function.name
        arguments=json.loads(tool_call.function.arguments)
        print(f"Tool Called : {tool_name}", flush=True)
        tool=globals().get(tool_name)
        result=tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})

    return results



In [16]:
reader=PdfReader("vignesh_linkedin.pdf")
linkedin=""
for page in reader.pages:
    text=page.extract_text()
    if text:
        linkedin+=text

with open("vigneshwar_kandhaiya_summary.txt", "r") as f:
    summary=f.read()

name="Vigneshwar Kandhaiya"

In [17]:
system_prompt=f""" You are acting as a {name} . You are answering questions on {name}'s profile.
Particularly, questions related to the {name}'s career, expectation,background and skills.
Your responsibility is to represent {name} for interactions on the website as faithfully as possible.
You are provided with a summary of {name}'s background and LinkedIn profile which you can use to answer the questions.
Be professional and engaging , as if you are talking to a potential client or a Future employer who wants to know about you.
If you don't know the answer to a question, use your record_unknown_question tool to record questions that you couldn't answer or even trivial or unrelated to career.
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

\n\n ## Summary : \n {summary} \n\n ## LinkedIn : \n {linkedin}
With this context , please chat with the user ,always staying in character as {name}
"""

In [19]:
def chat(message,history):
    messages=[{"role": "system","content": system_prompt}] + history + [{"role": "user","content":message}]
    done=False
    while not done:
        response=openai.chat.completions.create(model="gpt-4o-mini",messages=messages,tools=tools)
        finish_reason=response.choices[0].finish_reason

        if finish_reason=="tool_calls":
            message=response.choices[0].message
            tool_calls=message.tool_calls
            results=handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done=True
    return response.choices[0].message.content

In [22]:
gr.ChatInterface(chat,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool Called : record_unknown_question
Push: Recording question Do you like cricket? that I could not answer
Tool Called : record_user_details
Push: Recording the interest from user Name not provided with email vigneshwar.k2@gmail.com and notes Notes not provided
